## MODEL ARCHITECTURE FOR LIDAR AND CAMERA DATA
-----


### Overview



In [ ]:
# General import ->
import numpy as np
import matplotlib.image as mpimg
import cv2
import pickle

In [ ]:
# Model related import ->
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.layers import Flatten, Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.preprocessing import image

from keras.applications.vgg16 import VGG16

In [ ]:
# File import ->
import steering_angles
import model

In [ ]:
def get_model():
    # Bottom layers for reference ->
    pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=[48, 75, 3])
    x = pretrained_model.output
    x = Flatten()(x)

    # Add top layers ->
    x = Dense(3000, activation='relu', name='fc1', W_regularizer=l2(0.0001))(x)
    x = Dense(1000, activation='relu', name='fc2', W_regularizer=l2(0.0001))(x)
    x = Dense(500, activation='relu', name='fc2', W_regularizer=l2(0.0001))(x)
    x = Dropout(0.25)(x)
    predictions = Dense(1)(x)

    model = Model(input=pretrained_model.input, output=predictions)
    
    # Train top layers only ->
    for layer in pretrained_model.layers:
        layer.trainable = False
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

if __name__ == '__main__':
  
    # Get the training and validation data from pickle ->

    training_pickle = 'train.p'
    with open(training_data, 'rb') as handle:
        training_info = pickle.load(handle)

    validation_pickle = 'validation.p'
    with open(validation_pickle, 'rb') as handle:
        validation_info = pickle.load(handle)

    # Create model ->

    model = create_model()
    checkpoint = ModelCheckpoint(filepath='model-{epoch:02d}.h5')
    callback_list = [checkpoint]
    
    # Train the model ->
    model.fit_generator(
        get_steering_angle(training_info, batch_size=48),
        samples_per_epoch=1000, nb_epoch=30,
        validation=validation(validation_info), validation_nr=len(validation_info),
        callbacks=callback_list)

    # Save the model ->
    model.save('model.h5')
    print("The model is saved")

Beforre training the model you should read this -> https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c (step-by-step tutorial for training with the model...) and from this https://www.researchgate.net/post/How-to-choose-from-which-layer-to-start-unfreezing-pretrained-model (explains a bit how to start changing, i.e training, the model if using freezed weights...) + https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751 